### 2.3.1 wasted a day trying to solve hierachy problem with interactoins
Copilot just pretended to understand my intentions and sent me on a wild goose chase

In [ ]:
current_features = [set(), set(), set()]

In [ ]:
def stepwise_selection(X, y, 
                       initial_features=[], 
                       threshold_in=0.01, 
                       verbose=True):
    """
    Perform forward stepwise selection based on p-values from statsmodels OLS.

    Parameters:
    - X: DataFrame of candidate predictors
    - y: Series or array of target variable
    - initial_features: list of features to start with
    - threshold_in: p-value threshold for including a feature
    - verbose: whether to print progress

    Returns:
    - List of selected features
    """
    while True:
        changed = False
        # Candidates not yet included
        excluded = list(set(X.columns) - set(included))
        new_pvals = pd.Series(index=excluded, dtype=float)

        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            new_pvals[new_column] = model.pvalues[new_column]

        best_pval = new_pvals.min()
        if best_pval < threshold_in:
            best_feature = new_pvals.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f"Added {best_feature} with p-value {best_pval:.4f}")

        if not changed:
            break

    return included

num_order will fetch the order by taking a element from the set and returning 1 if primitive or number of terms in interaction

In [ ]:
def num_order(previous_order):
    term = next(iter(previous_order))
    return len(term) if isinstance(term, frozenset) else 1

this finds all permutations of the new_term (always present) and n-1 terms from the previous order

In [ ]:
from itertools import combinations

def generate_permutation_sets(new_term, previous_order):
    """
    new_term: frozenset[str] — candidate term of order n
    previous_order: set[frozenset[str]] — all valid terms of order n-1

    Returns: set[frozenset[frozenset[str]]] — each set includes new_term + (n-1) previous terms
    """
    if num_order(previous_order) == 1:
        return {frozenset([new_term])}

    candidates = set()
    for combo in combinations(previous_order, num_order(previous_order) - 1):
        #candidates = frozenset(combo + (new_term,))
        candidates.add(combo)
    permutation_sets = set()
    permutation_sets.update(candidates | frozenset(new_term))

    return permutation_sets

this retrieves primitive terms from permutations

In [ ]:
def num_order(previous_order):
    term = next(iter(previous_order))
    return len(term) if isinstance(term, frozenset) else 1

In [ ]:
# requeried second run to work on 2way
def extract_primitives(permutations):
    term = next(iter(permutations))
    primitives = set()
    if isinstance(term, frozenset):
        for perm in permutations:
            primitives.update(perm)
    else:
        for perm in permutations:
            primitives.add(perm)
    return primitives

this find a higher order term if valid

In [ ]:
def valid_next_order(new_term, previous_order):
    num_terms = len(previous_order)
    next_order = frozenset()
    if num_order(previous_order) <= num_terms:
        permutations = generate_permutation_sets(new_term, previous_order)
        current_order = previous_order | {new_term}
        if all(p.issubset(current_order) for p in permutations):
            # new_term is valid — all required ancestry present
            print(permutations)
            primative_terms = extract_primitives(permutations)
            return frozenset(primative_terms)
    return next_order

In [ ]:
def extract_primitives(permutations):
    primitives = set()
    for term in permutations:
        if isinstance(term, frozenset):
            primitives.update(term)
        else:
            primitives.add(term)
    return primitives

In [ ]:
previous_order = set()
previous_order.add(frozenset({'Month', 'Year'}))
previous_order.add(frozenset({'Month', 'cdi'}))
new_term = frozenset({'Year', 'cdi'})
valid_next_order(new_term, previous_order)

frozenset()

In [ ]:
valid_next_order('Month', {'Year'})

frozenset()

In [ ]:
valid_next_order('Month', {'Year', 'cdi'})

frozenset()

In [ ]:
previous_order = set()
previous_order.add('Month')
previous_order.add('Year')
previous_order.add('depth')
new_term = 'cdi'
valid_next_order(new_term, previous_order)

{frozenset({'cdi'})}


frozenset({'cdi'})

In [ ]:
num_terms = len(previous_order)
next_order = frozenset()
permutations = generate_permutation_sets(new_term, previous_order)
permutations

{frozenset({'Year', 'cdi'})}

In [ ]:
features

{'Month',
 'Year',
 'cdi',
 'depth',
 'dmin',
 'gap',
 'latitude',
 'longitude',
 'magnitude',
 'mmi',
 'nst',
 'sig'}